## example sound

WAV files: the waveform audio file format encodes a sampled sound wave

In [1]:
# Example: Sound

from wave import open
from struct import Struct
from math import floor

frame_rate = 11025

def encode(x):
    """Encode float x between -1 and 1 as two bytes.
    (See https://docs.python.org/3/library/struct.html)
    """
    i = int(16384 * x)
    return Struct('h').pack(i)

def play(sampler, name='song.wav', seconds=2):
    """Write the output of a sampler function as a wav file.
    (See https://docs.python.org/3/library/wave.html)
    """
    out = open(name, 'wb')
    out.setnchannels(1)
    out.setsampwidth(2)
    out.setframerate(frame_rate)
    t = 0
    while t < seconds * frame_rate:
        sample = sampler(t)
        out.writeframes(encode(sample))
        t = t + 1
    out.close()

def tri(frequency, amplitude=0.3):
    """A continuous triangle wave."""
    period = frame_rate // frequency
    def sampler(t):
        saw_wave = t / period - floor(t / period + 0.5)
        tri_wave = 2 * abs(2 * saw_wave) - 1
        return amplitude * tri_wave
    return sampler

In [2]:
c_freq, e_freq, g_freq = 261.63, 329.63, 392.00

In [3]:
c = tri(c_freq)
t = 0
while t < 100:
    print(c(t))
    t += 1
play(c)

-0.3
-0.2714285714285714
-0.24285714285714285
-0.21428571428571427
-0.18571428571428572
-0.15714285714285714
-0.1285714285714286
-0.1
-0.07142857142857144
-0.04285714285714287
-0.014285714285714301
0.014285714285714301
0.04285714285714284
0.07142857142857144
0.09999999999999998
0.1285714285714286
0.1571428571428571
0.18571428571428572
0.21428571428571425
0.24285714285714285
0.2714285714285714
0.3
0.2714285714285714
0.24285714285714277
0.21428571428571433
0.18571428571428572
0.1571428571428571
0.1285714285714285
0.10000000000000005
0.07142857142857144
0.04285714285714284
0.014285714285714235
-0.014285714285714235
-0.04285714285714284
-0.07142857142857144
-0.10000000000000005
-0.1285714285714285
-0.1571428571428571
-0.18571428571428572
-0.21428571428571433
-0.24285714285714277
-0.2714285714285714
-0.3
-0.2714285714285715
-0.24285714285714277
-0.21428571428571433
-0.18571428571428558
-0.1571428571428571
-0.12857142857142864
-0.09999999999999991
-0.07142857142857144
-0.04285714285714297
-0

In [4]:
def both(f, g):
    return lambda t: f(t) + g(t)

c = tri(c_freq)
e = tri(e_freq)
g = tri(g_freq)

play(both(c, e))

In [6]:
def note(f, start, end):
    """Play f for a fixed duration."""
    def sampler(t):
        seconds = t / frame_rate
        if seconds < start:
            return 0
        elif seconds > end:
            return 0
        else:
            return f(t)
    return sampler

play(both(note(c, 0, 1/4), note(e, 1/2, 1)))

In [7]:

def note(f, start, end, fade=.01):
    """Play f for a fixed duration."""
    def sampler(t):
        seconds = t / frame_rate
        if seconds < start:
            return 0
        elif seconds > end:
            return 0
        elif seconds < start + fade:
            return (seconds - start) / fade * f(t)
        elif seconds > end - fade:
            return (end - seconds) / fade * f(t)
        else:
            return f(t)
    return sampler

play(note(tri(e_freq), 1, 1.5))

In [8]:
low_g = tri(g_freq / 2)

play(both(note(e, 0, 1/8), note(low_g, 1/8, 3/8)))

play(both(note(c, 0, 1), both(note(e, 0, 1), note(g, 0, 1))))

In [9]:
def mario(c, e, g, low_g):
    z = 0
    song = note(e, z, z + 1/8)
    z += 1/8
    song = both(song, note(e, z, z + 1/8))
    z += 1/4
    song = both(song, note(e, z, z + 1/8))
    z += 1/4
    song = both(song, note(c, z, z + 1/8))
    z += 1/8
    song = both(song, note(e, z, z + 1/8))
    z += 1/4
    song = both(song, note(g, z, z + 1/4))
    z += 1/2
    song = both(song, note(low_g, z, z + 1/4))
    return song

def mario_at(octave):
    c = tri(octave * c_freq)
    e = tri(octave * e_freq)
    g = tri(octave * g_freq)
    low_g = tri(octave * g_freq / 2)
    return mario(c, e, g, low_g)

play(both(mario_at(1), mario_at(1/2)))